In [2]:
# import libraries
from gensim.models.fasttext import FastText
import gensim
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
import pandas as pd
from gensim.models.phrases import Phrases, Phraser
import numpy as np
import xgboost 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
tqdm.pandas(desc="progress-bar")
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
from collections import Counter
from nltk import ngrams

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
def classification_report(x_train, x_test, y_train, y_test):
  models = []
  models.append(('LogisticRegression', linear_model.LogisticRegression(solver='newton-cg',multi_class='multinomial')))
  models.append(('RandomForest', ensemble.RandomForestClassifier(n_estimators=100)))

  for name, model in models:
      clf=model
      clf.fit(x_train, y_train)
      y_pred=clf.predict(x_test)
      print(f"{name}:")
      print(f"accuracy: {metrics.accuracy_score(y_pred=y_pred, y_true=y_test)}")
      print(f"precision: {metrics.precision_score(y_pred=y_pred, y_true=y_test, average='macro')}")
      print(f"recall: {metrics.recall_score(y_pred=y_pred, y_true=y_test, average='macro')}")
      print(f"{metrics.classification_report(y_pred=y_pred, y_true=y_test)}")

In [4]:
def get_word_counts(data):
  words = data.tweet.to_string().split()
  return Counter(words)
  

In [5]:
# fastText hyper parameters
# sg = 0 cbow, 1 skip-gram
# min_count = corpusta kelimenin en az bulunma sayısı eğer kelime bu kadardan az geçiyorsa anlam ifade etmediği varsayılır(default=5)
# vector_size = kelimelerin ifade edileceği vektörün boyut sayısı
# window = current ve predicted word arasındaki maksimum mesafe
# loss = "ns" "hs" "softmax"
# negative = eğer sıfırdan büyük olursa negative sampling kullanılır 5-20 arasında olmalı
# alpha = başlangıç learning rate
# min_n: char ngram minimum uzunluğu default:3
# max_n: max length of char ngrams (Default 6)
# epoch = iterasyon sayısı


In [6]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [7]:
def train_fasText(corpus, n_epoch, name_corpus, sg, vector_size, negative, window, min_count, alpha, min_n, max_n):
  cores = multiprocessing.cpu_count()
  model = FastText(sg=sg, size=vector_size, negative=negative, window=window, min_count=min_count, workers=cores, alpha=alpha, min_n=min_n, max_n=max_n)
  model.build_vocab([x.words for x in tqdm(corpus)])

  for epoch in range(n_epoch):
    model.train(utils.shuffle([x.words for x in tqdm(corpus)]), total_examples=len(corpus), epochs=1)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

  model.save(f"/content/drive/MyDrive/hesaplamalı_anlambilim_ödev/trained_embeddings/fastText_{name_corpus}_sg_{sg}_size_{vector_size}_window_{window}_min_count_{min_count}.model")
  return model

In [8]:
def get_mean_vector(word2vec_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in word2vec_model.wv]
    if len(words) >= 1:
        return np.mean(word2vec_model[words], axis=0)
    else:
        return np.zeros((1, word2vec_model.vector_size))

In [9]:
def get_vectors(model, corpus):
  vectors = []
  for sentence in corpus:
      vec = get_mean_vector(model, sentence)
      vectors.append(vec)
  return vectors

In [10]:
def get_max_len_sentence(series):
  res = series.str.split().str.len().max()

  print(f"The maximum length in words are : {res}") 

**TWEET METINLERI İÇİN BENCHMARK**

In [11]:
tweet_train = pd.read_csv("/content/drive/MyDrive/hesaplamalı_anlambilim_ödev/preprocess_train.csv")
tweet_test = pd.read_csv("/content/drive/MyDrive/hesaplamalı_anlambilim_ödev/preprocess_test.csv")

In [12]:
tweet_test.dropna(inplace=True)
tweet_test.reset_index(drop=True,inplace=True)

tweet_train.dropna(inplace=True)
tweet_train.reset_index(drop=True,inplace=True)

x_train=tweet_train.tweet
y_train=tweet_train.sentiment.map({'olumlu':1,'olumsuz':-1,'notr':0}).values
x_test=tweet_test.tweet
y_test=tweet_test.sentiment.map({'olumlu':1,'olumsuz':-1,'notr':0}).values

In [13]:
concat = pd.concat([x_train, x_test])
corpus = labelize_tweets_ug(concat, 'all')

In [14]:
corpus_train = pd.DataFrame(x_train)['tweet'].apply(lambda x: x.split())
corpus_test = pd.DataFrame(x_test)['tweet'].apply(lambda x: x.split())

In [15]:
get_max_len_sentence(pd.DataFrame(concat).tweet)

The maximum length in words are : 27


# n_epoch = 25, sg = 1, negative = 5, alpha = 0.065
# değişenler:
# min_n, max_n = (2,4)  (3,6) (4,8) window= 3 vector_size= 150
# window 3 5 7 min,max = (4,8)
# vector size = 25 50 75 100 window = 3 min,max = (4,8)

# min_n, max_n = (2,4)

In [31]:
model = train_fasText(corpus=corpus, 
                        n_epoch=25, 
                        name_corpus="tweet", 
                        sg=1, 
                        negative=5,
                        alpha = 0.065,
                        min_n =2,
                        max_n = 4,
                        window = 3,
                        vector_size=150,
                        min_count=2)

vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)


classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 17222/17222 [00:00<00:00, 1362371.58it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.6487819025522041
precision: 0.6430791713300478
recall: 0.6346088512046438
              precision    recall  f1-score   support

          -1       0.67      0.77      0.72      1373
           0       0.64      0.59      0.61      1160
           1       0.62      0.55      0.58       915

    accuracy                           0.65      3448
   macro avg       0.64      0.63      0.64      3448
weighted avg       0.65      0.65      0.65      3448

RandomForest:
accuracy: 0.6374709976798144
precision: 0.6503638499540287
recall: 0.6122830677581264
              precision    recall  f1-score   support

          -1       0.62      0.83      0.71      1373
           0       0.65      0.56      0.60      1160
           1       0.69      0.44      0.54       915

    accuracy                           0.64      3448
   macro avg       0.65      0.61      0.62      3448
weighted avg       0.65      0.64      0.63      3448

SVM:
accuracy: 0.685034802784222

# min_n, max_n = (3,6)

In [32]:
model = train_fasText(corpus=corpus, 
                        n_epoch=25, 
                        name_corpus="tweet", 
                        sg=1, 
                        negative=5,
                        alpha = 0.065,
                        min_n =3,
                        max_n = 6,
                        window = 3,
                        vector_size=150,
                        min_count=2)

vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)


classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 17222/17222 [00:00<00:00, 1769364.45it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.652262180974478
precision: 0.6479713590702573
recall: 0.6376235530740754
              precision    recall  f1-score   support

          -1       0.67      0.77      0.72      1373
           0       0.65      0.59      0.62      1160
           1       0.63      0.55      0.59       915

    accuracy                           0.65      3448
   macro avg       0.65      0.64      0.64      3448
weighted avg       0.65      0.65      0.65      3448

RandomForest:
accuracy: 0.6531322505800464
precision: 0.6707176163641102
recall: 0.628402999277249
              precision    recall  f1-score   support

          -1       0.63      0.84      0.72      1373
           0       0.66      0.58      0.62      1160
           1       0.72      0.47      0.57       915

    accuracy                           0.65      3448
   macro avg       0.67      0.63      0.63      3448
weighted avg       0.66      0.65      0.64      3448

SVM:
accuracy: 0.6879350348027842


# min_n, max_n = (4,8)

In [33]:
model = train_fasText(corpus=corpus, 
                        n_epoch=25, 
                        name_corpus="tweet", 
                        sg=1, 
                        negative=5,
                        alpha = 0.065,
                        min_n =4,
                        max_n = 8,
                        window = 3,
                        vector_size=150,
                        min_count=2)

vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)


classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 17222/17222 [00:00<00:00, 1780529.55it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.6368909512761021
precision: 0.6336631485466095
recall: 0.6213164743729886
              precision    recall  f1-score   support

          -1       0.65      0.77      0.70      1373
           0       0.63      0.56      0.59      1160
           1       0.62      0.53      0.58       915

    accuracy                           0.64      3448
   macro avg       0.63      0.62      0.62      3448
weighted avg       0.64      0.64      0.63      3448

RandomForest:
accuracy: 0.6490719257540604
precision: 0.663095303564246
recall: 0.6238536294439593
              precision    recall  f1-score   support

          -1       0.63      0.84      0.72      1373
           0       0.65      0.57      0.61      1160
           1       0.71      0.46      0.56       915

    accuracy                           0.65      3448
   macro avg       0.66      0.62      0.63      3448
weighted avg       0.66      0.65      0.64      3448

SVM:
accuracy: 0.6830046403712297

# window 3

In [16]:
model = train_fasText(corpus=corpus, 
                        n_epoch=25, 
                        name_corpus="tweet", 
                        sg=1, 
                        negative=5,
                        alpha = 0.065,
                        min_n =4,
                        max_n = 18,
                        window = 3,
                        vector_size=150,
                        min_count=2)

vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)


classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 17222/17222 [00:00<00:00, 1751814.12it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.6496519721577726
precision: 0.645502872865643
recall: 0.6342557066521046
              precision    recall  f1-score   support

          -1       0.66      0.77      0.71      1373
           0       0.65      0.59      0.62      1160
           1       0.62      0.54      0.58       915

    accuracy                           0.65      3448
   macro avg       0.65      0.63      0.64      3448
weighted avg       0.65      0.65      0.65      3448

RandomForest:
accuracy: 0.6548723897911833
precision: 0.6724687404112669
recall: 0.630759174955842
              precision    recall  f1-score   support

          -1       0.63      0.84      0.72      1373
           0       0.67      0.59      0.62      1160
           1       0.73      0.47      0.57       915

    accuracy                           0.65      3448
   macro avg       0.67      0.63      0.64      3448
weighted avg       0.67      0.65      0.65      3448



# window 5

In [17]:
model = train_fasText(corpus=corpus, 
                        n_epoch=25, 
                        name_corpus="tweet", 
                        sg=1, 
                        negative=5,
                        alpha = 0.065,
                        min_n = 4,
                        max_n = 8,
                        window = 5,
                        vector_size=150,
                        min_count=2)

vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)


classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 17222/17222 [00:00<00:00, 1727266.94it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.6415313225058005
precision: 0.6393875416255268
recall: 0.6284215449616422
              precision    recall  f1-score   support

          -1       0.65      0.76      0.70      1373
           0       0.63      0.57      0.60      1160
           1       0.63      0.56      0.59       915

    accuracy                           0.64      3448
   macro avg       0.64      0.63      0.63      3448
weighted avg       0.64      0.64      0.64      3448

RandomForest:
accuracy: 0.6551624129930395
precision: 0.6750695717340665
recall: 0.6290173147797805
              precision    recall  f1-score   support

          -1       0.62      0.84      0.72      1373
           0       0.67      0.59      0.63      1160
           1       0.73      0.45      0.56       915

    accuracy                           0.66      3448
   macro avg       0.68      0.63      0.63      3448
weighted avg       0.67      0.66      0.65      3448



# window 7

In [18]:
model = train_fasText(corpus=corpus, 
                        n_epoch=25, 
                        name_corpus="tweet", 
                        sg=1, 
                        negative=5,
                        alpha = 0.065,
                        min_n = 4,
                        max_n = 8,
                        window = 7,
                        vector_size=150,
                        min_count=2)

vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)


classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 17222/17222 [00:00<00:00, 1461581.96it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.6316705336426914
precision: 0.6269684729508896
recall: 0.6161324786764658
              precision    recall  f1-score   support

          -1       0.65      0.76      0.70      1373
           0       0.62      0.56      0.59      1160
           1       0.61      0.52      0.56       915

    accuracy                           0.63      3448
   macro avg       0.63      0.62      0.62      3448
weighted avg       0.63      0.63      0.63      3448

RandomForest:
accuracy: 0.6447215777262181
precision: 0.6632324041858676
recall: 0.6202809787839475
              precision    recall  f1-score   support

          -1       0.62      0.83      0.71      1373
           0       0.64      0.58      0.61      1160
           1       0.73      0.46      0.56       915

    accuracy                           0.64      3448
   macro avg       0.66      0.62      0.63      3448
weighted avg       0.66      0.64      0.64      3448



# vector size 25

In [19]:
model = train_fasText(corpus=corpus, 
                        n_epoch=25, 
                        name_corpus="tweet", 
                        sg=1, 
                        negative=5,
                        alpha = 0.065,
                        min_n = 4,
                        max_n = 8,
                        window = 3,
                        vector_size=25,
                        min_count=2)

vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)


classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 17222/17222 [00:00<00:00, 1595632.95it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.6197795823665894
precision: 0.6142664843949519
recall: 0.6020096003423312
              precision    recall  f1-score   support

          -1       0.63      0.77      0.70      1373
           0       0.62      0.53      0.57      1160
           1       0.59      0.50      0.54       915

    accuracy                           0.62      3448
   macro avg       0.61      0.60      0.60      3448
weighted avg       0.62      0.62      0.61      3448

RandomForest:
accuracy: 0.660092807424594
precision: 0.663147722575612
recall: 0.6408168738097012
              precision    recall  f1-score   support

          -1       0.65      0.80      0.72      1373
           0       0.67      0.61      0.64      1160
           1       0.67      0.51      0.58       915

    accuracy                           0.66      3448
   macro avg       0.66      0.64      0.65      3448
weighted avg       0.66      0.66      0.65      3448



# vector size 50

In [20]:
model = train_fasText(corpus=corpus, 
                        n_epoch=25, 
                        name_corpus="tweet", 
                        sg=1, 
                        negative=5,
                        alpha = 0.065,
                        min_n = 4,
                        max_n = 8,
                        window = 5,
                        vector_size=50,
                        min_count=2)

vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)


classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 17222/17222 [00:00<00:00, 1915876.81it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.6281902552204176
precision: 0.623084895384813
recall: 0.6124655910993159
              precision    recall  f1-score   support

          -1       0.64      0.76      0.70      1373
           0       0.64      0.56      0.60      1160
           1       0.59      0.52      0.55       915

    accuracy                           0.63      3448
   macro avg       0.62      0.61      0.61      3448
weighted avg       0.63      0.63      0.62      3448

RandomForest:
accuracy: 0.6592227378190255
precision: 0.671673799363108
recall: 0.6372324731273994
              precision    recall  f1-score   support

          -1       0.64      0.83      0.72      1373
           0       0.67      0.59      0.63      1160
           1       0.71      0.49      0.58       915

    accuracy                           0.66      3448
   macro avg       0.67      0.64      0.64      3448
weighted avg       0.67      0.66      0.65      3448



# vector size 75

In [21]:
model = train_fasText(corpus=corpus, 
                        n_epoch=25, 
                        name_corpus="tweet", 
                        sg=1, 
                        negative=5,
                        alpha = 0.065,
                        min_n = 4,
                        max_n = 8,
                        window = 5,
                        vector_size=75,
                        min_count=2)

vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)


classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 17222/17222 [00:00<00:00, 1662546.11it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.627030162412993
precision: 0.6228882545948219
recall: 0.6128147179539472
              precision    recall  f1-score   support

          -1       0.64      0.76      0.69      1373
           0       0.63      0.55      0.58      1160
           1       0.60      0.54      0.57       915

    accuracy                           0.63      3448
   macro avg       0.62      0.61      0.61      3448
weighted avg       0.63      0.63      0.62      3448

RandomForest:
accuracy: 0.6534222737819025
precision: 0.6697048766782502
recall: 0.6291239295917255
              precision    recall  f1-score   support

          -1       0.63      0.84      0.72      1373
           0       0.66      0.58      0.62      1160
           1       0.72      0.47      0.57       915

    accuracy                           0.65      3448
   macro avg       0.67      0.63      0.63      3448
weighted avg       0.66      0.65      0.64      3448



# vector size 100

In [22]:
model = train_fasText(corpus=corpus, 
                        n_epoch=25, 
                        name_corpus="tweet", 
                        sg=1, 
                        negative=5,
                        alpha = 0.065,
                        min_n = 4,
                        max_n = 8,
                        window = 5,
                        vector_size=100,
                        min_count=2)

vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)


classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 17222/17222 [00:00<00:00, 1799200.55it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.6415313225058005
precision: 0.6377774169307019
recall: 0.6267648249850534
              precision    recall  f1-score   support

          -1       0.65      0.77      0.71      1373
           0       0.63      0.57      0.60      1160
           1       0.62      0.54      0.58       915

    accuracy                           0.64      3448
   macro avg       0.64      0.63      0.63      3448
weighted avg       0.64      0.64      0.64      3448

RandomForest:
accuracy: 0.6528422273781903
precision: 0.6713334355549536
recall: 0.6291323927185452
              precision    recall  f1-score   support

          -1       0.62      0.84      0.71      1373
           0       0.66      0.58      0.62      1160
           1       0.73      0.48      0.58       915

    accuracy                           0.65      3448
   macro avg       0.67      0.63      0.64      3448
weighted avg       0.66      0.65      0.64      3448



# en iyiler vector_size = 100, window=5, min,max = (3,6)

In [23]:
model = train_fasText(corpus=corpus, 
                        n_epoch=25, 
                        name_corpus="tweet", 
                        sg=1, 
                        negative=5,
                        alpha = 0.065,
                        min_n = 3,
                        max_n = 6,
                        window = 5,
                        vector_size=100,
                        min_count=2)

vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)


classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 17222/17222 [00:00<00:00, 1866664.17it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.6453016241299304
precision: 0.6400547229655147
recall: 0.6289731910100733
              precision    recall  f1-score   support

          -1       0.66      0.78      0.72      1373
           0       0.64      0.57      0.61      1160
           1       0.62      0.53      0.57       915

    accuracy                           0.65      3448
   macro avg       0.64      0.63      0.63      3448
weighted avg       0.64      0.65      0.64      3448

RandomForest:
accuracy: 0.6499419953596288
precision: 0.6663915291036622
recall: 0.6245697460691407
              precision    recall  f1-score   support

          -1       0.62      0.84      0.72      1373
           0       0.66      0.57      0.61      1160
           1       0.72      0.46      0.56       915

    accuracy                           0.65      3448
   macro avg       0.67      0.62      0.63      3448
weighted avg       0.66      0.65      0.64      3448

